<a href="https://colab.research.google.com/github/EmelyanovAndreyNSK/PythonTasks/blob/master/FactoryQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Распределительная задача. 

Завод может производить $Y$ видов продукции. На продукцию есть покупатель, которому требуется определенное количество каждого вида: $m_i$ 

Для каждого вида продукции он либо покупает требуемое количество, либо не покупает совсем, если мы произвели недостаточно. При этом известна цена контракта на каждый вид продукции: $c_i$ 

Завод одновременно может выпускать только один вид продукции со скоростью: $p_i$

Требуется написать математическую модель и найти оптимальный план производства на $S$ суток, если известно, что переход от одной продукции к другой происходит мгновенно, а его стоимость $R$ рублей.


Запишем математическую модель.

Введем переменные:

$ x_{i} = \left\{
\begin{array}{ll}
          1, & \mbox { если выполняем контракт на $i$ вид продукции,}\\
          0, & \mbox { в противном случае. } \\
\end{array} \right. $

Договоримся о входных данных:

$Y$ - множество видов продукции;

$m_i$ - необходимый объем $i$ продукции для выполнения контракта;

$p_i$ - скорость производства $i$ продукции;

$c_i$ - стоимость контракта на $i$ продукцию.

Математическая модель:

Целевая функция:

$$\sum\limits_{i = 0}^{I} c_i x_i - R((\sum\limits_{i = 0}^{I} x_i)-1) \to \max_{x}$$

Ограничения:

$$x_i \in \mathbb B, \forall i \in I $$

$$\sum\limits_{i = 0}^{I} (m_i /p_i) x_i \leqslant S $$


In [1]:
!pip install pulp
!pip install cplex

     |████████████████████████████████| 40.6MB 100kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for amply: filename=amply-0.1.2-cp36-none-any.whl size=16573 sha256=97ad464485ba4451ec0d37c262c241fb34ff095bf853378602874f23077053be
  Stored in directory: /root/.cache/pip/wheels/84/18/f7/e5c3ed13ed5bb721763f77d4a924331d59ef115ce61c9d26eb
Successfully built amply
     |████████████████████████████████| 31.0MB 103kB/s 


In [2]:
import numpy as np
import time
from random import randrange
import pulp as plp

In [12]:
Y = 3 
R = 1 
S = 10
m = [10, 20, 30]
p = [25, 30, 35]
c = [15, 20, 25]

In [31]:
def signum (a):
  if a >=1:
    return 1
  else: 
    return 0

problem = plp.LpProblem(name='FactoryProblem', sense=plp.LpMaximize)
x  = {i : plp.LpVariable(cat=plp.LpInteger, name="x"+str(i)) for i in range(Y)}

problem.addConstraint(plp.LpConstraint(e=plp.lpSum((m[i]/p[i]) * x[i] for i in range(Y)),
                     sense=plp.LpConstraintLE,
                     rhs=S ,
                     name="constraint_{1}"))

problem.setObjective(plp.lpSum(c[i] * x[i] for i in range(Y))-R*(plp.lpSum(signum(x[i]) for i in range(Y))-1))
print(problem)

FactoryProblem:
MAXIMIZE
15*x0 + 20*x1 + 25*x2 + -2
SUBJECT TO
constraint_{1}: 0.4 x0 + 0.666666666667 x1 + 0.857142857143 x2 <= 10

VARIABLES
x0 free Integer
x1 free Integer
x2 free Integer



In [32]:
solver_list = plp.list_solvers()
print(solver_list)

['GLPK_CMD', 'PYGLPK', 'CPLEX_CMD', 'CPLEX_PY', 'CPLEX_DLL', 'GUROBI', 'GUROBI_CMD', 'MOSEK', 'XPRESS', 'PULP_CBC_CMD', 'COIN_CMD', 'COINMP_DLL', 'CHOCO_CMD', 'PULP_CHOCO_CMD', 'MIPCL_CMD', 'SCIP_CMD']


In [33]:
def experiment(name, solver):
    start = time.time()
    problem.solve(solver)
    answer = plp.value(problem.objective)
    print(f"{name} solved {time.time() - start}  answer : {answer}")
    
#experiment('Gurobi', plp.get_solver('GUROBI', msg=False))
#experiment('CPLEX', plp.get_solver('CPLEX_PY', msg=False))
#experiment('GLPK', plp.get_solver('GLPK_CMD', path="C:\GRB\winglpk-4.65\glpk-4.65\w64\glpsol.exe", msg=False))
experiment('PULP', plp.get_solver('PULP_CBC_CMD', msg=False))

PULP solved 0.0169980525970459  answer : 373.0


In [34]:
for v in problem.variables():
            print(v.name, "=", v.varValue)

x0 = 25.0
x1 = 0.0
x2 = 0.0


In [35]:
def experiment(name, solver, timeLimit):
    times = []
    start = time.time()
    current = start
    while current - start <= timeLimit:
        problem.solve(solver)
        startIter, current = current, time.time()
        times.append(current - startIter)
    
    answer = plp.value(problem.objective)
    print(f"{name} solved {len(times) - 1} instances "
          f"in {timeLimit} seconds answer : {answer}")

    
#experiment('Gurobi', plp.get_solver('GUROBI', msg=False), 120)
#experiment('CPLEX', plp.get_solver('CPLEX_PY', msg=False), 120)
#experiment('GLPK', plp.get_solver('GLPK_CMD', path="C:\GRB\winglpk-4.65\glpk-4.65\w64\glpsol.exe", msg=False), 120)
experiment('PULP', plp.get_solver('PULP_CBC_CMD', msg=False), 120)

PULP solved 12590 instances in 120 seconds answer : 373.0
